<a href="https://colab.research.google.com/github/Chuchu-1234/ML-Projects/blob/main/AM_En_Translation/Aug_03_16k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Installation**

In [3]:
!pip install torchdata mosestokenizer sentencepiece torch

In [4]:
!pip install pip==24.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 17.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [5]:
!pip install fairseq

  Using cached fairseq-0.12.2.tar.gz (9.6 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached hydra_core-1.0.7-py3-none-any.whl.metadata (3.7 kB)
  Using cached omegaconf-2.0.6-py3-none-any.whl.metadata (3.0 kB)
  Using cached sacrebleu-2.4.3-py3-none-any.whl.metadata (51 kB)
  Using cached bitarray-2.9.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (34 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.3/288.3 kB 20.1 MB/s eta 0:00:00
  Created wheel for fairseq: filename=fairseq-0.12.2-cp310-cp310-linux_x86_64.whl size=11288617 sha256=1cb2

In [ ]:
!pip cache purge

In [6]:
!pip install sacrebleu sacremoses wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 94.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.0/311.0 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.7 MB/s eta 0:00:00
DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


## **Import and Mount**

In [7]:
import os
import random
import subprocess
import sacrebleu
import torch
import sentencepiece as spm
import re
from sacremoses import MosesTokenizer
from fairseq.data import Dictionary
from fairseq.models.transformer import TransformerModel
from sacrebleu import corpus_bleu
import wandb

In [9]:
#mount file
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
%cd /content
!git clone https://github.com/google/sentencepiece.git
%cd sentencepiece
!git checkout v0.1.97
!mkdir build
%cd build
!cmake ..
!make -j $(nproc)

/content
Cloning into 'sentencepiece'...
remote: Enumerating objects: 5387, done.
remote: Counting objects: 100% (2419/2419), done.
remote: Compressing objects: 100% (489/489), done.
remote: Total 5387 (delta 2041), reused 2081 (delta 1915), pack-reused 2968 (from 1)
Receiving objects: 100% (5387/5387), 28.08 MiB | 14.06 MiB/s, done.
Resolving deltas: 100% (3694/3694), done.
/content/sentencepiece
Note: switching to 'v0.1.97'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at 58f256c Updated the document
/content

## **Enable Graphics**

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print(f"GPU device found. Using {torch.cuda.get_device_name(device)}.")

    # Limit GPU memory allocation
    gpu_memory_fraction = 0.3  # Set the desired fraction of GPU memory to be allocated
    torch.cuda.set_per_process_memory_fraction(gpu_memory_fraction)
    print(f"GPU memory limit set to {gpu_memory_fraction * 100}% of available memory.")
else:
    device = "cpu"
    print("GPU device not found. Using CPU.")

GPU device found. Using Tesla T4.
GPU memory limit set to 30.0% of available memory.


## **File Path**

In [11]:
src_file = '/content/drive/My Drive/Colab Notebooks/datasets/Amharic_English/train.am-en.base.am'
tgt_file = '/content/drive/My Drive/Colab Notebooks/datasets/Amharic_English/train.am-en.base.en'
src_test = '/content/drive/MyDrive/Colab Notebooks/datasets/Amharic_English/test.am-en.base.am'
tgt_test = '/content/drive/MyDrive/Colab Notebooks/datasets/Amharic_English/test.am-en.base.en'
src_val = '/content/drive/MyDrive/Colab Notebooks/datasets/Amharic_English/dev.am-en.base.am'
tgt_val = '/content/drive/MyDrive/Colab Notebooks/datasets/Amharic_English/dev.am-en.base.en'

### **Make Directory**

In [12]:
%cd /content
!mkdir clean
%cd /content
!mkdir processed
%cd /content
!mkdir duplicate
%cd /content
!mkdir normal
%cd /content
!mkdir spm
%cd /content

/content
/content
/content
/content
/content
/content


## **Cleaning and Filtering**

### **Clean**
*   Only do this for **train** and **val** files



In [13]:
PATH = '/content/drive/MyDrive/Colab\ Notebooks/datasets/Amharic_English'
TOOLS = '/content/drive/MyDrive/Colab\ Notebooks'
CLEAN_PATH = '/content/clean'
PROCESSED_PATH = '/content/processed'
REMOVE_DUP = '/content/duplicate'
NORMAL = '/content/normal'

In [14]:
!ls -l $TOOLS/mosesdecoder/scripts/training/clean-corpus-n.perl

-rw------- 1 root root 4257 Aug 14  2023 '/content/drive/MyDrive/Colab Notebooks/mosesdecoder/scripts/training/clean-corpus-n.perl'


In [15]:
!chmod +x $TOOLS/mosesdecoder//scripts/training/clean-corpus-n.perl

In [16]:
!$TOOLS/mosesdecoder/scripts/training/clean-corpus-n.perl -max-word-length 50 $PATH/train.am-en.base am en $CLEAN_PATH/train.clean 0 150

clean-corpus.perl: processing /content/drive/MyDrive/Colab Notebooks/datasets/Amharic_English/train.am-en.base.am & .en to /content/clean/train.clean, cutoff 0-150, ratio 9
..........(100000)....
Input sentences: 140000  Output sentences:  139981


In [17]:
!$TOOLS/mosesdecoder/scripts/training/clean-corpus-n.perl -max-word-length 50 $PATH/dev.am-en.base am en $CLEAN_PATH/dev.clean 0 150

clean-corpus.perl: processing /content/drive/MyDrive/Colab Notebooks/datasets/Amharic_English/dev.am-en.base.am & .en to /content/clean/dev.clean, cutoff 0-150, ratio 9

Input sentences: 2864  Output sentences:  2864


### **Preprocess**

#### **Preprocess**

In [18]:
!ls -l $TOOLS/preprocess/preprocess/build/bin/simple_cleaning
!chmod +x $TOOLS/preprocess/preprocess/build/bin/simple_cleaning

-rw------- 1 root root 413408 Aug 13  2023 '/content/drive/MyDrive/Colab Notebooks/preprocess/preprocess/build/bin/simple_cleaning'


In [19]:
!chmod +x $TOOLS/preprocess/preprocess/build/bin/dedupe

In [20]:
!$TOOLS/preprocess/preprocess/build/bin/simple_cleaning -p $CLEAN_PATH/train.clean.am $CLEAN_PATH/train.clean.en $PROCESSED_PATH/train.clean.pp.am $PROCESSED_PATH/train.clean.pp.en

Reading /content/clean/train.clean.am
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Kept 114449 / 139981 = 0.817604


In [21]:
!$TOOLS/preprocess/preprocess/build/bin/simple_cleaning -p $CLEAN_PATH/dev.clean.am $CLEAN_PATH/dev.clean.en $PROCESSED_PATH/dev.clean.pp.am $PROCESSED_PATH/dev.clean.pp.en

Reading /content/clean/dev.clean.am
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Kept 2361 / 2864 = 0.824372


In [22]:
!$TOOLS/preprocess/preprocess/build/bin/dedupe -p $PROCESSED_PATH/train.clean.pp.am $PROCESSED_PATH/train.clean.pp.en $REMOVE_DUP/train.clean.pp.dedup.am $REMOVE_DUP/train.clean.pp.dedup.en

Reading /content/processed/train.clean.pp.am
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Kept 114224 / 114449 = 0.998034


In [23]:
!$TOOLS/preprocess/preprocess/build/bin/dedupe -p $PROCESSED_PATH/dev.clean.pp.am $PROCESSED_PATH/dev.clean.pp.en $REMOVE_DUP/dev.clean.pp.dedup.am $REMOVE_DUP/dev.clean.pp.dedup.en

Reading /content/processed/dev.clean.pp.am
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Kept 2361 / 2361 = 1


### **Normalization**

In [24]:
!sacremoses normalize < $REMOVE_DUP/train.clean.pp.dedup.am > $NORMAL/train.clean.pp.dedup.norm.am
!sacremoses normalize < $REMOVE_DUP/train.clean.pp.dedup.en > $NORMAL/train.clean.pp.dedup.norm.en

In [25]:
!sacremoses normalize < $REMOVE_DUP/dev.clean.pp.dedup.am > $NORMAL/dev.clean.pp.dedup.norm.am
!sacremoses normalize < $REMOVE_DUP/dev.clean.pp.dedup.en > $NORMAL/dev.clean.pp.dedup.norm.en

In [26]:
!sacremoses normalize < $PATH/test.am-en.base.am > $NORMAL/test.clean.pp.dedup.norm.am

### **Tokenization**

#### Concatinate train data

In [27]:
!cat $NORMAL/train.clean.pp.dedup.norm.am $NORMAL/train.clean.pp.dedup.norm.en > $NORMAL/train.clean.pp.dedup.norm.am-en

#### Train SentencePiece

In [28]:
SPM = '/content/spm'

In [29]:
%cd /content

/content


In [30]:
!/content/sentencepiece/build/src/spm_train -input=$NORMAL/train.clean.pp.dedup.norm.am -model_prefix=$SPM/am.32kspm -vocab_size=16000 -bos_id=0 -pad_id=1 -eos_id=2 -unk_id=3 -split_by_whitespace=true

sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: /content/normal/train.clean.pp.dedup.norm.am
  input_format: 
  model_prefix: /content/spm/am.32kspm
  model_type: UNIGRAM
  vocab_size: 16000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 3
  bos_id: 0
  eos_id: 2
  pad_id: 1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
  enable_differential_privacy: 0
  differential_priva

In [31]:
!/content/sentencepiece/build/src/spm_train -input=$NORMAL/train.clean.pp.dedup.norm.en -model_prefix=$SPM/en.32kspm -vocab_size=16000 -bos_id=0 -pad_id=1 -eos_id=2 -unk_id=3 -split_by_whitespace=true

sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: /content/normal/train.clean.pp.dedup.norm.en
  input_format: 
  model_prefix: /content/spm/en.32kspm
  model_type: UNIGRAM
  vocab_size: 16000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 3
  bos_id: 0
  eos_id: 2
  pad_id: 1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
  enable_differential_privacy: 0
  differential_priva

#### SentencePiece Encode

In [32]:
!/content/sentencepiece/build/src/spm_encode -help

sentencepiece

Usage: /content/sentencepiece/build/src/spm_encode [options] files

   --help (show help)  type: bool default: false
   --version (show version)  type: bool default: false
   --minloglevel (Messages logged at a lower level than this don't actually get logged anywhere)  type: int default: 0
   --model (model file name)  type: std::string default: ""
   --output_format (choose from piece, id, proto, nbest_piece, nbest_id, or nbest_proto)  type: std::string default: "piece"
   --input (input filename)  type: std::string default: ""
   --output (output filename)  type: std::string default: ""
   --extra_options (':' separated encoder extra options, e.g., "reverse:bos:eos")  type: std::string default: ""
   --nbest_size (NBest size)  type: int32 default: 10
   --alpha (Smoothing parameter for sampling mode.)  type: double default: 0.5
   --random_seed (Seed value for random generator.)  type: uint32 default: 4294967295
   --vocabulary (Restrict the vocabulary. The encoder onl

In [33]:
!/content/sentencepiece/build/src/spm_encode -model=$SPM/am.32kspm.model < $NORMAL/train.clean.pp.dedup.norm.am > $SPM/train.clean.pp.dedup.norm.spm32k.am
!/content/sentencepiece/build/src/spm_encode -model=$SPM/en.32kspm.model < $NORMAL/train.clean.pp.dedup.norm.en > $SPM/train.clean.pp.dedup.norm.spm32k.en

In [34]:
def encode_file(spm_model_file, input_file, output_file):
    sp = spm.SentencePieceProcessor(model_file=spm_model_file)
    with open(input_file, 'r') as input_file, open(output_file, 'w') as output_file:
        for line in input_file:
            encoded_line = sp.encode(line, out_type=str, enable_sampling=True, alpha=0.1, nbest_size=-1)
            output_file.write(' '.join(map(str, encoded_line)) + '\n')
            #encoded = sp.encode_as_pieces(line, enable_sampling=True, alpha=0.1, nbest_size=-1)
            #output_file.write(" ".join(encoded) + "\n")

In [35]:
#/content/spm/am.32kspm.model
encode_file('/content/spm/am.32kspm.model', '/content/normal/train.clean.pp.dedup.norm.am', '/content/spm/train.clean.pp.dedup.norm.spm32k.am')
encode_file('/content/spm/en.32kspm.model', '/content/normal/train.clean.pp.dedup.norm.en', '/content/spm/train.clean.pp.dedup.norm.spm32k.en')
encode_file('/content/spm/am.32kspm.model', '/content/normal/dev.clean.pp.dedup.norm.am', '/content/spm/dev.clean.pp.dedup.norm.spm32k.am')
encode_file('/content/spm/en.32kspm.model', '/content/normal/dev.clean.pp.dedup.norm.en', '/content/spm/dev.clean.pp.dedup.norm.spm32k.en')
encode_file('/content/spm/am.32kspm.model', '/content/normal/test.clean.pp.dedup.norm.am', '/content/spm/test.clean.pp.dedup.norm.spm32k.am')

In [36]:
!/content/sentencepiece/build/src/spm_encode -model=$SPM/am.32kspm.model < $NORMAL/dev.clean.pp.dedup.norm.am > $SPM/dev.clean.pp.dedup.norm.spm32k.am
!/content/sentencepiece/build/src/spm_encode -model=$SPM/en.32kspm.model < $NORMAL/dev.clean.pp.dedup.norm.en > $SPM/dev.clean.pp.dedup.norm.spm32k.en

In [37]:
!/content/sentencepiece/build/src/spm_encode -model=$SPM/am.32kspm.model < $NORMAL/test.clean.pp.dedup.norm.am > $SPM/test.clean.pp.dedup.norm.spm32k.am

### **Fairseq Preprocess**

In [38]:
%cd '/content'

/content


In [39]:
!cut -f1 $SPM/am.32kspm.vocab | tail -n +5 | sed "s/$/ 100/g" > dict.am.txt
!cut -f1 $SPM/en.32kspm.vocab | tail -n +5 | sed "s/$/ 100/g" > dict.en.txt

In [ ]:
!fairseq-preprocess \
    --trainpref $SPM/train.clean.pp.dedup.norm.spm32k \
    --validpref $SPM/dev.clean.pp.dedup.norm.spm32k \
    --destdir "bin" \
    --joined-dictionary \
    --srcdict "dict.txt" \
    --source-lang "am" \
    --target-lang "en" \
    --bpe sentencepiece \
    --workers 10

2023-08-22 06:23:41.144381: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-08-22 06:23:43 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2023-08-22 06:23:44 | INFO | fairseq_cli.preprocess | Namespace(no_progress_bar=False, log_interval=100, log_format=None, log_file=None, aim_repo=None, aim_run_hash=None, tensorboard_logdir=None, wandb_project=None, azureml_logging=False, seed=1, cpu=False, tpu=False, bf16=False, memory_efficient_bf16=False, fp16=False, memory_efficient_fp16=False, fp16_no_flatten_grads=False, fp16_init_scale=128, fp16_scale_window=None, fp16_scale_tolerance=0.0, on_cpu_convert_precision=False, min_loss_scale=0.0001, threshold_loss_scale=None, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, user_dir=None, empty_cache_freq=0, all_gather_list_size=16384, model_parallel_size=1, quantization_config_path=None, profile=False, reset_logging=False, s

In [40]:
!fairseq-preprocess \
    --trainpref $SPM/train.clean.pp.dedup.norm.spm32k \
    --validpref $SPM/dev.clean.pp.dedup.norm.spm32k \
    --destdir "bin" \
    --srcdict "dict.am.txt"\
    --tgtdict "dict.en.txt"\
    --source-lang "am" \
    --target-lang "en" \
    --bpe sentencepiece \
    --tokenizer "moses" \
    --workers 10
#--joined-dictionary \

2024-10-04 13:18:37.785644: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-04 13:18:37.822180: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-04 13:18:37.834570: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-04 13:18:39.536131: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-10-04 13:18:42 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2024-10-04 13:18:44 | INFO | fairseq_cli.preprocess | Namespace(no_progress_bar=False, log_interval=100, log_format=None, log_f

### **Fairseq Train**

In [ ]:
wandb.login()
# e9cc01f037e1659b06953e66410a32cf7e404b7f

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
!fairseq-train \
        "bin" \
        --fp16 \
        --arch transformer_wmt_en_de \
        --optimizer adam \
        --adam-betas '(0.9, 0.98)' \
        --clip-norm 0.0 \
        --lr 5e-4 \
        --lr-scheduler inverse_sqrt \
        --warmup-updates 4000 \
        --warmup-init-lr 1e-07 \
        --dropout 0.3 \
        --weight-decay 0.0 \
        --criterion label_smoothed_cross_entropy \
        --label-smoothing 0.1 \
        --save-dir "model_output" \
        --log-format json \
        --log-interval 100 \
        --max-tokens 8000 \
        --max-epoch 100 \
        --patience 5 \
        --seed 3921 \
        --eval-bleu \
        --eval-bleu-args '{"beam": 5, "max_len_a": 1.2, "max_len_b": 10}' \
        --eval-bleu-detok space \
        --eval-bleu-remove-bpe sentencepiece \
        --eval-bleu-print-samples \
        --best-checkpoint-metric bleu \
        --maximize-best-checkpoint-metric \
        --keep-interval-updates 4000 \
        --wandb-project "7_wandb" \
        --no-epoch-checkpoints
#--share-all-embeddings

2023-08-29 12:34:22.813000: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-08-29 12:34:24 | INFO | numexpr.utils | NumExpr defaulting to 2 threads.
2023-08-29 12:34:25 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2023-08-29 12:34:27 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': 'json', 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': '7_wandb', 'azureml_logging': False, 'seed': 3921, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_ini

In [ ]:
model_output = 'model_output'

In [ ]:
!fairseq-preprocess \
    --testpref $NORMAL/train.clean.pp.dedup.norm.spm8k \
    --destdir "bin" \
    --joined-dictionary \
    --srcdict "dict.txt"\
    --target-lang "en" \
    --workers 8

In [ ]:
!fairseq-generate bin \
    --path $model_output/checkpoint_best.pt \
    --batch-size 128 \
    --beam 5

2023-08-03 16:31:30.476766: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-08-03 16:31:33 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2023-08-03 16:31:35 | INFO | fairseq_cli.generate | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'a